In [5]:
import pandas as pd

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [7]:
!kaggle datasets download -d mohamedhanyyy/chest-ctscan-images

Dataset URL: https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images
License(s): ODbL-1.0
 87% 103M/119M [00:00<00:00, 276MB/s] 
100% 119M/119M [00:00<00:00, 249MB/s]


In [8]:
import zipfile
zip_ref = zipfile.ZipFile('/content/chest.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [9]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten
from keras.utils import image_dataset_from_directory
from keras.applications.vgg16 import VGG16

In [10]:
base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150, 150, 3)
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
base.trainable = False

In [12]:
model = Sequential()
model.add(base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [13]:
train_ds = image_dataset_from_directory(
    directory='/content/Data/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150)
)

Found 613 files belonging to 4 classes.


In [14]:
validation_ds = image_dataset_from_directory(
    directory='/content/Data/valid',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150)
)

Found 72 files belonging to 4 classes.


In [15]:
def process(image, label):
  image = tensorflow.cast(image/255., tensorflow.float32)
  return (image, label)

train_ds = train_ds.map(process)
validation_ds =  validation_ds.map(process)

In [16]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(train_ds, epochs=5, validation_data=validation_ds)

Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 0.9982 - loss: 0.0227 - val_accuracy: 0.8472 - val_loss: 0.4012
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - accuracy: 0.9991 - loss: 0.0151 - val_accuracy: 0.8750 - val_loss: 0.3035
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.9993 - loss: 0.0106 - val_accuracy: 0.9306 - val_loss: 0.2835
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.9986 - loss: 0.0145 - val_accuracy: 0.9167 - val_loss: 0.3093
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.9973 - loss: 0.0185 - val_accuracy: 0.8889 - val_loss: 0.3545
